In [1]:
# for database connections
from sqlalchemy import create_engine, inspect, text

# data
import pandas as pd

# Postgres credentials
import config as cfg

In [2]:
# Setup the SQL connection variables
SQL_USERNAME = cfg.SQL_USERNAME
SQL_PASSWORD = cfg.SQL_PASSWORD
SQL_IP = cfg.SQL_IP
SQL_PORT = cfg.SQL_PORT
DATABASE = cfg.DATABASE

In [3]:
# Connect to PostgreSQL server
connection_string = f'postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{SQL_PORT}/{DATABASE}'
engine = create_engine(connection_string)

## upload campaign, category, subcategory and contact tables

In [4]:
# Check to see if the category and subcategory tables are in the database
inspector = inspect(engine)
print(inspector.get_table_names())

OperationalError: (psycopg2.OperationalError) FATAL:  database "economic_atlas_rural_america" does not exist

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
# Display all the columns of the tables in PostgreSQL to make sure they have the correct columns prior to loading the data

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Using the inspector to print the column names within each table and its types
for table in tables:
    print(f"Table name: {table}")
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])
        
    print()

In [ ]:
# Open category csv file and read it into a pandas dataframe
category_df = pd.read_csv('DB_Input/category.csv')
category_df.head()

In [ ]:
# Open subcategory csv file and read it into a pandas dataframe
subcategory_df = pd.read_csv('DB_Input/subcategory.csv')
subcategory_df.head()

In [ ]:
# Open contact csv file and read it into a pandas dataframe
contact_df = pd.read_csv('DB_Input/contact.csv')
contact_df.head()

In [ ]:
# Open campaign csv file and read it into a pandas dataframe
campaign_df = pd.read_csv('DB_Input/campaign.csv')
campaign_df.head()

In [ ]:
# If category table exists in the database, load the category data into the table
if 'category' in inspector.get_table_names():
    category_df.to_sql('category', schema='public', con=engine, index=False, if_exists='append', method='multi')

In [ ]:
# If subcategory table exists in the database, load the subcategory data into the table
if 'subcategory' in inspector.get_table_names():
    subcategory_df.to_sql('subcategory', schema='public', con=engine, index=False, if_exists='append', method='multi')

In [ ]:
# If contact table exists in the database, load the contact data into the table
if 'contact' in inspector.get_table_names():
    contact_df.to_sql('contact', schema='public', con=engine, index=False, if_exists='append', method='multi')

In [ ]:
# If campaign table exists in the database, load the campaign data into the table
if 'campaign' in inspector.get_table_names():
    campaign_df.to_sql('campaign', schema='public', con=engine, index=False, if_exists='append', method='multi')

In [ ]:
# Verify count from campaign table after load
query = """
		SELECT
			count(*)
		FROM
			campaign
		;
		"""

campaign_count_after_import_df = pd.read_sql(text(query), con=engine)
campaign_count_after_import_df.head()

In [ ]:
# Verify count from category table after load
query = """
		SELECT
			count(*)
		FROM
			category
		;
		"""

category_count_after_import_df = pd.read_sql(text(query), con=engine)
category_count_after_import_df.head()

In [ ]:
# Verify count from subcategory table after load
query = """
		SELECT
			count(*)
		FROM
			subcategory
		;
		"""

subcategory_count_after_import_df = pd.read_sql(text(query), con=engine)
subcategory_count_after_import_df.head()

In [ ]:
# Verify count from contact table after load
query = """
		SELECT
			count(*)
		FROM
			contact
		;
		"""

contact_count_after_import_df = pd.read_sql(text(query), con=engine)
contact_count_after_import_df.head()